# Random Forest Pipeline

Steps:
1. generate featuresets and targets from psuedosynthetic timeseries: 
2. generate featuresets and targets from ambient timeseries.
3. re-map JGR:SE Dittmann, et al 2022 featuresets into aligned featuresets for validation
4. nested cross validation of random forest model using psuedosynthetic velocities
5. generate overall model for use with validation from real data

### 1. generate featuresets and targets from psuedosynthetic timeseries:
2000+ waveforms x 5 augmentation noise levels per/waveform

In [ ]:
%run ../bin/features/ts_2_featuresets.py

### 2. generate featuresets and targets from ambient timeseries:
first 30 minutes of 1507 hours of 5Hz spread across network from 2017-2023.

In [ ]:
%run ../bin/features/ambient_2_fs.py

second 30 minutes preserved for ambient testing

In [ ]:
%run ../bin/features/ambient_2_fs_testing.py

### 3. re-map JGR:SE Dittmann, et al 2022 featuresets into aligned featuresets for validation

In [ ]:
%run ../bin/features/jgr_fs_2_new.py

### 4. nested cross validation of random forest model using psuedosynthetic velocities

depending on number of hyperparameters tested and compute nodes, this takes a long time.

**best run as "nohangup"**   *eg:* `nohup python -u nest_xval_synth_MP.py > program.out 2>&1 &`

In [ ]:
%run ../bin/models/nest_xval_synth_MP.py

### 5. generate overall model for use with validation from real data

In [ ]:
%run ../bin/models/complete_model_synth.py